# Использование RePlay
Данный ноутбук показывает базовый порядок работы с библиотекой.
- загрузка датасета
- деление на трейн и тест
- обучение модели
- сравнение с бейзлайном

Документацию можно собрать из папки `docs` командой `make html`. Она создаст папку `_build` с документацией -- `/docs/_build/html/index.html`

In [1]:
%config Completer.use_jedi = False

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

Целевая архитектура библиотеки - ЛД 3.0, поэтому используем Spark.
Для отладки моделей (не на кластере) создаём локальную сессию.

Объект `State` позволяет использовать одну и ту же сессию Spark в разных объектах.
Если сессия уже инициализирована, то её нужно передать при инициализации `State`. Модули библиотеки, которым необходимо использовать спарк-сессию, например для конвертации из пандас в спарк, будут искать сессию именно в `State`.

По умолчанию создастся дефолтная сессия. Простой способ получить сессию с заданным количеством выделенной памяти -- функция `get_spark_session` модуля `session_handler`.

In [3]:
from replay.session_handler import State

spark = State().session
spark

In [4]:
K = 10

## 0. Подготовка данных <a name='data-preparator'></a>
Популярные датасеты для рекомендательных систем можно найти в библиотеке rs_datasets.

Данные датасета доступны в виде `pandas.DataFrame` атрибутов объекта.
Посмотреть доступные данные можно с помощью метода `info`.

In [5]:
from rs_datasets import MovieLens

data = MovieLens("1m")
data.info()

ratings


,user_id,item_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968



users


,user_id,gender,age,occupation,zip_code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117



items


,item_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


### 0.1. DataPreparator

Внутренний формат данных в библиотеке -- spark dataframe.
Модели replay ожидают на вход pandas или spark dataframe, содержащий обязательные столбцы item_id, user_id. Для некоторых моделей обязательно наличие столбца relevance, содержащего оценки релевантности объектов для пользователей.
Для разбиения данных по времени с помощью Splitter и фильтров в логе должен присутсововать столбец timestamp.

В replay есть DataPreparator, который автоматически конвертирует pandas и spark dataframe c логом в нужный формат (переименование/создание обязательных столбцов, корректное чтение дат, проверка отсутствия пропусков). DataPreparetor может конвертировать dataframe с признаками пользователей/объектов (нужно указать один из столбцов, соответсвующий user_id/item_id.

Также конвертировать pandas dataframes в spark можно с помощью вспомогательной функции convert_to_spark.

In [6]:
from replay.data_preparator import DataPreparator

log = DataPreparator().transform(
    data=data.ratings,
    columns_names={
        "user_id": "user_id",
        "item_id": "item_id",
        "relevance": "rating",
        "timestamp": "timestamp"
    }
)

In [7]:
log.show(3)

+-------+-------+---------+-------------------+
|user_id|item_id|relevance|          timestamp|
+-------+-------+---------+-------------------+
|      1|   1193|      5.0|2000-12-31 22:12:40|
|      1|    661|      3.0|2000-12-31 22:35:09|
|      1|    914|      3.0|2000-12-31 22:32:48|
+-------+-------+---------+-------------------+
only showing top 3 rows



In [8]:
from replay.utils import convert2spark
items = convert2spark(data.users)

### 0.2. Split

Библиотека содержит различные схемы валидации рекомендательных систем, встречающиеся в литературе.

`UserSplitter` отбирает для теста некоторое количество или долю объектов для каждого пользователя.

In [9]:
from replay.splitters import UserSplitter

splitter = UserSplitter(
    drop_cold_items=True,
    drop_cold_users=True,
    item_test_size=1,
    user_test_size=500,
    seed=1234,
    shuffle=True
)
train, test = splitter.split(log)
(
    train.count(), 
    test.count()
)

(999709, 500)

## 1. Обучение модели 

#### SLIM
Один из простых, но эффективных алгоритмов 

In [10]:
from replay.models import SLIM

slim = SLIM(lambda_=0.01, beta=0.3)

In [11]:
%%time

slim.fit(log=train)

29-Jan-21 07:55:07, replay, DEBUG: Начало обучения SLIM
DEBUG:replay:Начало обучения SLIM
29-Jan-21 07:55:07, replay, DEBUG: Предварительная стадия обучения (pre-fit)
DEBUG:replay:Предварительная стадия обучения (pre-fit)
29-Jan-21 07:55:11, replay, DEBUG: Основная стадия обучения (fit)
DEBUG:replay:Основная стадия обучения (fit)


CPU times: user 2.39 s, sys: 0 ns, total: 2.39 s
Wall time: 14.1 s


In [12]:
%%time

recs = slim.predict(
    k=K,
    users=test.select('user_id').distinct(),
    items=test.select('item_id').distinct(),
    log=train,
    filter_seen_items=True
)

29-Jan-21 07:55:21, replay, DEBUG: Начало предикта SLIM
DEBUG:replay:Начало предикта SLIM


CPU times: user 995 ms, sys: 53 ms, total: 1.05 s
Wall time: 15.2 s


## 2. Оценка качества и сравнение результатов моделей

В библиотеке реализованы различные метрики качества рекомендательных систем, встречающихся в литературе.
Их можно использовать напрямую, либо запоминать результаты с помощью класса `Experiment`.

In [13]:
from replay.metrics import HitRate, NDCG, MAP
from replay.experiment import Experiment

metrics = Experiment(test, {NDCG(): K,
                            MAP() : K,
                            HitRate(): [1, int(K/2), K]})


In [14]:
%%time
metrics.add_result("SLIM", recs)
metrics.results

CPU times: user 104 ms, sys: 40.2 ms, total: 144 ms
Wall time: 50.1 s


,HitRate@1,HitRate@5,HitRate@10,MAP@10,NDCG@10
SLIM,0.09,0.23,0.342,0.150803,0.195098


## 3. Примеры использования других моделей RePlay

#### ALS
Библиотека также содержит классические алгоритмы рекомендаций, например, матричную факторизацию

In [15]:
from replay.models import ALSWrap

als = ALSWrap(rank=100)

In [16]:
%%time
als.fit(log=train)

29-Jan-21 07:56:26, replay, DEBUG: Начало обучения ALSWrap
DEBUG:replay:Начало обучения ALSWrap
29-Jan-21 07:56:26, replay, DEBUG: Предварительная стадия обучения (pre-fit)
DEBUG:replay:Предварительная стадия обучения (pre-fit)
29-Jan-21 07:56:27, replay, DEBUG: Основная стадия обучения (fit)
DEBUG:replay:Основная стадия обучения (fit)


CPU times: user 309 ms, sys: 17.4 ms, total: 327 ms
Wall time: 33.1 s


In [17]:
%%time
recs = als.predict(
    k=K,
    users=test.select('user_id').distinct(),
    items=test.select('item_id').distinct(),
    log=train,
    filter_seen_items=True
)

29-Jan-21 07:57:00, replay, DEBUG: Начало предикта ALSWrap
DEBUG:replay:Начало предикта ALSWrap


CPU times: user 772 ms, sys: 177 ms, total: 949 ms
Wall time: 28.6 s


In [18]:
%%time
metrics.add_result("ALS", recs)
metrics.results

CPU times: user 91.6 ms, sys: 21.7 ms, total: 113 ms
Wall time: 33.7 s


,HitRate@1,HitRate@5,HitRate@10,MAP@10,NDCG@10
SLIM,0.09,0.230,0.342,0.150803,0.195098
ALS,0.10,0.262,0.404,0.178067,0.230724


#### MultVAE 
Пример использования DL в рекомендациях

In [39]:
from replay.models import MultVAE

multvae = MultVAE(epochs=500)

In [40]:
%%time
multvae.fit(log=train)

29-Jan-21 08:04:33, replay, DEBUG: Начало обучения MultVAE
DEBUG:replay:Начало обучения MultVAE
29-Jan-21 08:04:33, replay, DEBUG: Предварительная стадия обучения (pre-fit)
DEBUG:replay:Предварительная стадия обучения (pre-fit)
29-Jan-21 08:04:34, replay, DEBUG: Основная стадия обучения (fit)
DEBUG:replay:Основная стадия обучения (fit)
29-Jan-21 08:04:36, replay, DEBUG: Составление батча:
DEBUG:replay:Составление батча:
29-Jan-21 08:04:36, replay, DEBUG: Обучение модели
DEBUG:replay:Обучение модели
29-Jan-21 08:04:37, replay, DEBUG: Epoch[1] current loss: 1353.87424
DEBUG:replay:Epoch[1] current loss: 1353.87424
29-Jan-21 08:04:37, replay, DEBUG: Epoch[1] validation average loss: 1478.64697
DEBUG:replay:Epoch[1] validation average loss: 1478.64697
29-Jan-21 08:04:38, replay, DEBUG: Epoch[2] current loss: 1248.28292
DEBUG:replay:Epoch[2] current loss: 1248.28292
29-Jan-21 08:04:38, replay, DEBUG: Epoch[2] validation average loss: 1478.83655
DEBUG:replay:Epoch[2] validation average loss:

CPU times: user 7min 2s, sys: 5min 33s, total: 12min 35s
Wall time: 1min 37s


In [41]:
%%time

recs = multvae.predict(
    k=10,
    users=test.select('user_id').distinct(),
    items=test.select('item_id').distinct(),
    log=train,
    filter_seen_items=True
)

29-Jan-21 08:06:11, replay, DEBUG: Начало предикта MultVAE
DEBUG:replay:Начало предикта MultVAE
29-Jan-21 08:06:17, replay, DEBUG: Предсказание модели
DEBUG:replay:Предсказание модели
/home/volodkevich/replay_tasks/venv376/lib/python3.7/site-packages/torch/storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)


CPU times: user 808 ms, sys: 130 ms, total: 938 ms
Wall time: 23 s


In [42]:
%%time
metrics.add_result("MultVAE", recs)
metrics.results

CPU times: user 105 ms, sys: 24.8 ms, total: 129 ms
Wall time: 41.5 s


,HitRate@1,HitRate@5,HitRate@10,MAP@10,NDCG@10
SLIM,0.090,0.230,0.342,0.150803,0.195098
ALS,0.100,0.262,0.404,0.178067,0.230724
MultVAE,0.014,0.070,0.150,0.043444,0.067716
my_model,0.006,0.054,0.094,0.026181,0.041819


## 4 Сравнение результатов различных моделей
С помощью experiment можно сравнить качество моделей, построенных с использованием различных библиотек.
Чтобы сравнение было корректным, нужно использовать одинаковое разбиение данных. 

### 4.1 Экспортируем train

In [43]:
train.toPandas().to_csv("train.csv", index=False)

In [44]:
!head -n 5 train.csv

user_id,item_id,relevance,timestamp
1024,2396,4.0,2000-11-23 18:18:26
1024,25,4.0,2000-11-23 18:19:17
1024,1210,1.0,2000-11-23 18:19:17
1024,2020,5.0,2000-11-23 18:18:26


#### 4.2 Обучаем модель и получаем рекомендации в формате `id пользователя - id объекта - relevance`

Предположим, что это произошло и у нас есть рекомендации в виде csv-файла. Ниже в качестве пример используем рекомендации, полученные одной из моделей, с рандомными релевантностями.

In [45]:
from pyspark.sql.functions import rand

In [46]:
recs.withColumn('relevance', rand(seed=123)).toPandas().to_csv("recs.csv", index=False)

Теперь нужно прочитать рекомендации в формате, поддерживаемом библиотекой

In [47]:
recs = DataPreparator().transform(
    path="recs.csv",
    columns_names={
        "user_id": "user_id",
        "item_id": "item_id",
        "relevance": "relevance"
    },
    header=True,
    format_type="csv"
)

#### 4.3 Сравним качество внешней модели с предыдущими результатами

In [48]:
metrics.add_result("my_model", recs)
metrics.results.sort_values("NDCG@10", ascending=False)

,HitRate@1,HitRate@5,HitRate@10,MAP@10,NDCG@10
ALS,0.100,0.262,0.404,0.178067,0.230724
SLIM,0.090,0.230,0.342,0.150803,0.195098
MultVAE,0.014,0.070,0.150,0.043444,0.067716
my_model,0.006,0.054,0.094,0.026181,0.041819
